In [11]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import datetime
from pudb import set_trace;

with open('data/countries_s1_dbg') as f:
    facts = f.read().splitlines()
facts = [el.split(',') for el in facts]
preds = [fact[0] for fact in facts]
subjs = [fact[1] for fact in facts]
objs = [fact[2] for fact in facts]

unique = sorted(list(set(preds)))
num_unique = len(unique)
num_predicates = num_unique
predsToIdx = dict(zip(unique,range(num_unique)))
idxToPreds = dict(zip(range(num_unique),unique))

unique = sorted(list(set(subjs+objs)))
num_unique = len(unique)
num_constants = num_unique
consToIdx = dict(zip(unique,range(num_unique)))
idxToCons = dict(zip(range(num_unique),unique))

facts = np.array([(predsToIdx[preds[i]],consToIdx[subjs[i]], consToIdx[objs[i]]) for i in range(len(facts))])
data = np.zeros((num_predicates,num_constants,num_constants))

#data: idx0->predicate, idx1->subj, idx2->obj
data[facts[:,0],facts[:,1],facts[:,2]] = 1
no_facts = int(np.sum(data))
data = torch.from_numpy(data)

predicates = torch.eye(num_predicates)
constants = torch.eye(num_constants)

knowledge_pos = (data==1).nonzero()
data_aux = knowledge_pos
knowledge_pos = torch.cat((predicates[knowledge_pos[:,0]],
                           constants[knowledge_pos[:,1]],
                           constants[knowledge_pos[:,2]]),dim=1)
num_facts = knowledge_pos.size()[0]
num_feats_per_fact = knowledge_pos.size()[1]
print(num_facts,num_feats_per_fact)

#helps removing repeated predicted facts -> same embeddings and constants, probably different scores
#this is of complexity K^3, could be optimized
def leaveTopK(preds,K):
    _,idx = torch.sort(preds[:,-1],descending=True)
    preds = preds[idx,:]
    out = preds[0,:].unsqueeze(0)
    for i in range(1,K):
        t = preds[i,:].unsqueeze(0)
        m,_ = torch.max(F.cosine_similarity(t[:,:-1].repeat(out.size()[0],1),out[:,:-1]),dim=0)
        if m<1:
            out = torch.cat((out,t),dim=0)
    return out

####FORWARD CHAINING
#input: some facts that can either be ground or predicted in previous steps
#output: predicted facts in this specific step (outer loop must gather all of them)
#computes the predictions of applying each rule to the facts, giving a score to each of them.
#leaves only topK scoring fact for each applied rule (not for the whole thing)
def forward_step(facts):
    #rule 1
    # b1(x,y)<-b1(y,x)
    rule_expanded = rules[0].expand(facts[:,:num_predicates].size())
    preds_r1 = F.cosine_similarity(rule_expanded,facts[:,:num_predicates],dim=1)
    preds_r1 = preds_r1*facts[:,-1]
    preds_r1 = preds_r1.unsqueeze(1)
    preds_r1 = torch.cat((rule_expanded,
                         facts[:,num_predicates+num_constants:-1],
                         facts[:,num_predicates:num_predicates+num_constants],
                         preds_r1),dim=1)
    preds_r1 = leaveTopK(preds_r1,K)
    #rule 2
    # b1(x,y)<-b2(x,z),b2(z,y)
    body1 = facts.repeat((1,num_facts)).view(-1,num_feats_per_fact+1)
    body2 = facts.repeat((num_facts,1))
    rule_expanded = rules[1].repeat(body1.size()[0],1)
    #previous scores
    preds_r2 = body1[:,-1]*body2[:,-1]
    #predicate of body1 with predicate of rule
    preds_r2 = preds_r2*F.cosine_similarity(rule_expanded[:,num_predicates:],body1[:,:num_predicates],dim=1)
    #predicate of body2 with predicate of rule
    preds_r2 = preds_r2*F.cosine_similarity(rule_expanded[:,num_predicates:],body2[:,:num_predicates],dim=1)
    #similarity between shared constants
    preds_r2 = preds_r2*F.cosine_similarity(body1[:,num_predicates+num_constants:-1],
                                            body2[:,num_predicates:num_predicates+num_constants],dim=1)
    preds_r2 = preds_r2.unsqueeze(1)
    preds_r2 = torch.cat((rule_expanded[:,:num_predicates]
                         ,body1[:,num_predicates:num_predicates+num_constants]
                         ,body2[:,num_predicates+num_constants:-1]
                         ,preds_r2)
                        ,dim=1)
    #removing repeated facts and leaving ones with highest score
    preds_r2 = leaveTopK(preds_r2,K)
    out = torch.cat((preds_r1,preds_r2),dim=0)
    return out

    






5 12


In [21]:
####TRAINING
#added params
# core_rel = Variable(knowledge_pos[[0,1,4]])
# target = Variable(knowledge_pos[[2,3]])

core_rel = Variable(knowledge_pos)
target = Variable(knowledge_pos)

num_iters = 200
learning_rate = .01
drop=0

steps = 1
num_rules = 2
epsilon=.001

K = 5 ##For top K

#rules should be:
#r1(x,y) <- r1(y,x)
#r1(x,y) <- r2(x,z),r2(z,x)
rules = [Variable(torch.rand(num_predicates), requires_grad=True),
         Variable(torch.rand(2*num_predicates), requires_grad=True)]

optimizer = torch.optim.Adam([
        {'params': rules}], 
        lr = learning_rate)

criterion = torch.nn.MSELoss(size_average=False)

for epoch in range(num_iters):
    # ##sampling
    # core_rel = torch.randperm(no_facts)
    # target = core_rel[no_samples:]
    # core_rel = core_rel[:no_samples]

    # core_rel = Variable(knowledge_pos[core_rel])
    # target = Variable(knowledge_pos)
    optimizer.zero_grad()
    facts = torch.cat((core_rel, Variable(torch.ones(core_rel.size()[0], 1))), 1)
    #will accumulate predictions separately to compare with target facts
    consequences = forward_step(facts)
    for step in range(1,steps):
        tmp = torch.cat((consequences,facts),dim=0)
        tmp = forward_step(tmp)
        consequences = torch.cat((consequences,tmp),dim=0)

    loss = 0
    for targ in target:
        _, indi = torch.max(F.cosine_similarity(targ.view(1,-1).expand(consequences[:,:-1].size()),consequences[:,:-1]),0)
        indi=indi.data[0]
        loss += criterion(consequences[indi,:-1],targ)+(1-(consequences[indi,-1]))
        #remove fact from predicted facts
        # if indi==0:
        #     facts = facts[1:,:]
        # elif indi+1 == facts.size()[0]:
        #     facts = facts[:indi,:]
        # else:
        #     facts = torch.cat((facts[:indi,:],facts[indi+1:,:]),dim=0)

    print(epoch, 'losssssssssssssssssssss',loss.data[0])
    print(rules)
    loss.backward()
    optimizer.step()


    

/Users/aldo.pareja/miniconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/aldo.pareja/miniconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 losssssssssssssssssssss tensor(8.0640)
[tensor([ 0.5830,  0.6683]), tensor([ 0.2406,  0.0174,  0.4055,  0.1823])]
1 losssssssssssssssssssss tensor(8.0082)
[tensor([ 0.5730,  0.6583]), tensor([ 0.2506,  0.0074,  0.4155,  0.1723])]
2 losssssssssssssssssssss tensor(7.9562)
[tensor([ 0.5631,  0.6483]), tensor([ 0.2606, -0.0017,  0.4254,  0.1623])]
3 losssssssssssssssssssss tensor(7.9079)
[tensor([ 0.5534,  0.6384]), tensor([ 0.2706, -0.0081,  0.4353,  0.1523])]
4 losssssssssssssssssssss tensor(7.8632)
[tensor([ 0.5440,  0.6284]), tensor([ 0.2806, -0.0107,  0.4450,  0.1424])]
5 losssssssssssssssssssss tensor(7.8220)
[tensor([ 0.5350,  0.6186]), tensor([ 0.2905, -0.0100,  0.4545,  0.1326])]
6 losssssssssssssssssssss tensor(7.7841)
[tensor([ 0.5268,  0.6088]), tensor([ 0.3005, -0.0075,  0.4637,  0.1229])]
7 losssssssssssssssssssss tensor(7.7494)
[tensor([ 0.5195,  0.5991]), tensor([ 0.3104, -0.0039,  0.4726,  0.1133])]
8 losssssssssssssssssssss tensor(7.7175)
[tensor([ 0.5132,  0.5896]), te

131 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0022,  0.9976]), tensor([ 1.0004e+00,  1.2322e-05,  6.1040e-01, -6.9384e-05])]
132 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0021,  0.9978]), tensor([ 1.0006e+00,  3.3036e-06,  6.1040e-01, -9.4446e-05])]
133 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0019,  0.9980]), tensor([ 1.0007e+00, -7.4916e-06,  6.1040e-01, -1.1314e-04])]
134 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0018,  0.9982]), tensor([ 1.0008e+00, -1.1222e-05,  6.1040e-01, -1.2528e-04])]
135 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0017,  0.9984]), tensor([ 1.0009e+00, -5.5335e-06,  6.1040e-01, -1.3097e-04])]
136 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0016,  0.9986]), tensor([ 1.0010e+00,  4.0915e-06,  6.1041e-01, -1.3057e-04])]
137 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0015,  0.9987]), tensor([ 1.0011e+00,  9.4589e-06,  6.1041e-01, -1.2465e-04])]
138 losssssssssssssssssssss tensor(4.0000)
[tensor([-0.0014,  

199 losssssssssssssssssssss tensor(4.)
[tensor([ 4.9109e-05,  1.0001e+00]), tensor([ 9.9999e-01,  3.7942e-07,  6.1041e-01, -2.5679e-06])]


tensor([[ 0.7627,  0.1373,  0.8353,  0.3743],
        [ 0.7627,  0.1373,  0.8353,  0.3743]])